In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../data/expanded_data_with_OSM.csv', sep = ',')

In [3]:
#два пропущенных региона - это южный федеральный округ
df['regions'] = df['regions'].replace(np.nan, 'Southern Federal District')

In [13]:
#for column in df.columns:
#    if column[0:8]=='distance':
#df[column]= df[column]*-1

In [14]:
#добавляю таргет переменную
df2 = pd.read_csv('C:/Users/User/Desktop/ВШЭ/Курсы/Годовой проект/Данные/train.csv')

In [15]:
df_train = df.merge(df2[['id','target']], left_on = 'id', right_on = 'id')

In [16]:
df_train.head()

,id,atm_group,address,address_rus,lat,lng,test_train_flag,geometry,distance_to_fast_food,distance_to_clothes,...,distance_to_airport,distance_to_parking,distance_to_parking_underground,cnt_apart_200m,cnt_banks_200m,cnt_atm_200m,cities,regions,states,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,train,POINT (142.738319 46.940994999999994),1101.997929,815.369715,...,39774.216642,2892.577916,2346.129844,9,0,0,Yuzhno-Sakhalinsk,Far Eastern Federal District,Sakhalin Oblast,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,train,POINT (142.75334799999996 46.937353),1177.665235,141.531360,...,40364.561477,2767.793411,2124.799118,17,0,0,Yuzhno-Sakhalinsk,Far Eastern Federal District,Sakhalin Oblast,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,train,POINT (142.741113 46.959413),536.421857,1054.650591,...,36765.157465,1815.884777,1864.222802,4,1,0,Yuzhno-Sakhalinsk,Far Eastern Federal District,Sakhalin Oblast,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,train,POINT (37.515146 55.805827),10.006280,18.646221,...,73227.945390,106.652808,471.901686,14,5,4,Moscow,Central Federal District,Moscow,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,train,POINT (88.211228 69.343541),1264.514620,2159.536521,...,NaN,5763.343722,5842.299373,13,0,0,Norilsk,Siberian Federal District,Krasnoyarsk Krai,0.079277


# Обучение моделей

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df_train.isna().sum()

id                                    0
atm_group                             0
address                               0
address_rus                           0
lat                                   0
lng                                   0
test_train_flag                       0
geometry                              0
distance_to_fast_food               490
distance_to_clothes                 503
distance_to_vending_parking        4554
distance_to_cafe                    218
distance_to_pharmacy                222
distance_to_atm                     334
distance_to_cinema                 1182
distance_to_shoe_shop               817
distance_to_bank                    155
distance_to_supermarket             222
distance_to_restaurant              571
distance_to_mobile_phone_shop       167
distance_to_convenience             144
distance_to_vending_any            2066
distance_to_bank_Сбербанк           549
distance_to_bank_ВТБ               1422
distance_to_bank_Альфа-Банк        2196


In [8]:
#замения пропуски максимальными значениями расстояний
df_train = df.copy()
df_train = df_train.fillna(df.max())

C:\Users\User\AppData\Local\Temp/ipykernel_7636/3399169565.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train = df_train.fillna(df.max())


In [20]:
df_train.columns

Index(['id', 'atm_group', 'address', 'address_rus', 'lat', 'lng',
       'test_train_flag', 'geometry', 'distance_to_fast_food',
       'distance_to_clothes', 'distance_to_vending_parking',
       'distance_to_cafe', 'distance_to_pharmacy', 'distance_to_atm',
       'distance_to_cinema', 'distance_to_shoe_shop', 'distance_to_bank',
       'distance_to_supermarket', 'distance_to_restaurant',
       'distance_to_mobile_phone_shop', 'distance_to_convenience',
       'distance_to_vending_any', 'distance_to_bank_Сбербанк',
       'distance_to_bank_ВТБ', 'distance_to_bank_Альфа-Банк',
       'distance_to_bank_Росбанк', 'distance_to_atm_Сбербанк',
       'distance_to_atm_ВТБ', 'distance_to_atm_Альфа-Банк',
       'distance_to_atm_Росбанк', 'distance_to_retail',
       'distance_to_residential', 'distance_to_office',
       'distance_to_commercial', 'distance_to_detached',
       'distance_to_train_station', 'distance_to_apartments',
       'distance_to_house', 'distance_to_railway_halt',
    

In [21]:
#все числовые переменные
df_train_numeric = df_train[['atm_group','distance_to_fast_food',
       'distance_to_clothes', 'distance_to_vending_parking',
       'distance_to_cafe', 'distance_to_pharmacy', 'distance_to_atm',
       'distance_to_cinema', 'distance_to_shoe_shop', 'distance_to_bank',
       'distance_to_supermarket', 'distance_to_restaurant',
       'distance_to_mobile_phone_shop', 'distance_to_convenience',
       'distance_to_vending_any', 'distance_to_bank_Сбербанк',
       'distance_to_bank_ВТБ', 'distance_to_bank_Альфа-Банк',
       'distance_to_bank_Росбанк', 'distance_to_atm_Сбербанк',
       'distance_to_atm_ВТБ', 'distance_to_atm_Альфа-Банк',
       'distance_to_atm_Росбанк', 'distance_to_retail',
       'distance_to_residential', 'distance_to_office',
       'distance_to_commercial', 'distance_to_detached',
       'distance_to_train_station', 'distance_to_apartments',
       'distance_to_house', 'distance_to_railway_halt',
       'distance_to_tram_stop', 'distance_to_bus_stop',
       'distance_to_railway_station', 'distance_to_airport',
       'distance_to_parking', 'distance_to_parking_underground'
       ,'cnt_apart_200m', 'cnt_banks_200m', 'cnt_atm_200m'
                            ]]

In [22]:
#таргет переменная 
df_train['target'].head()

0    0.011500
1    0.029710
2    0.009540
3   -0.094035
4    0.079277
Name: target, dtype: float64

# Просто регрессия на числовых признаках

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df_train_numeric, df_train['target'], test_size=0.25, random_state=42)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [25]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [26]:
y_pred = model.predict(X_test)

In [27]:
r2_score(y_test, y_pred)

0.3246826076123548

In [28]:
rms = mean_squared_error(y_test, y_pred)**0.5

In [29]:
rms

0.07095802106467328

# Масштабирование признаков

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:

scaler = StandardScaler() 
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = [list(X_train.columns)])
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = [list(X_test.columns)])

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [33]:
y_pred = model.predict(X_test)

In [34]:
r2_score(y_test, y_pred)

0.3252218474518983

In [35]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test, y_pred)**0.5

In [36]:
rms

0.07092968547485298

Особо качество не изменилось

# Пробую L1 регуляризацию (Lasso)

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
model = Lasso()
# your code here

In [38]:
lasso_params = {'alpha':list(np.arange(0.01, 1, 0.01))}


In [39]:
gscv = GridSearchCV(Lasso(), param_grid = lasso_params, cv = 10, scoring='r2').fit(X_train, y_train)

In [40]:
gscv.best_score_

0.2669440358475437

In [41]:
gscv.best_estimator_

Lasso(alpha=0.01)

In [42]:
model_l = Lasso(alpha = 0.01)
model_l.fit(X_train, y_train)

Lasso(alpha=0.01)

model_l = Lasso(alpha = 0.01)
model_l.fit(X_train, y_train)

In [43]:
y_pred = model_l.predict(X_test)

In [44]:
rms = mean_squared_error(y_test, y_pred)**0.5
rms

0.07314892864802164

Качество с L1 регуляризацией стало хуже

# Добавляю категориальные фичи

In [45]:
#все числовые переменные
df_train_numeric = df_train[['atm_group','distance_to_fast_food',
       'distance_to_clothes', 'distance_to_vending_parking',
       'distance_to_cafe', 'distance_to_pharmacy', 'distance_to_atm',
       'distance_to_cinema', 'distance_to_shoe_shop', 'distance_to_bank',
       'distance_to_supermarket', 'distance_to_restaurant',
       'distance_to_mobile_phone_shop', 'distance_to_convenience',
       'distance_to_vending_any', 'distance_to_bank_Сбербанк',
       'distance_to_bank_ВТБ', 'distance_to_bank_Альфа-Банк',
       'distance_to_bank_Росбанк', 'distance_to_atm_Сбербанк',
       'distance_to_atm_ВТБ', 'distance_to_atm_Альфа-Банк',
       'distance_to_atm_Росбанк', 'distance_to_retail',
       'distance_to_residential', 'distance_to_office',
       'distance_to_commercial', 'distance_to_detached',
       'distance_to_train_station', 'distance_to_apartments',
       'distance_to_house', 'distance_to_railway_halt',
       'distance_to_tram_stop', 'distance_to_bus_stop',
       'distance_to_railway_station', 'distance_to_airport',
       'distance_to_parking', 'distance_to_parking_underground'
       ,'cnt_apart_200m', 'cnt_banks_200m', 'cnt_atm_200m', 'regions'
                            ]]

In [46]:
df_train_numeric['atm_group'] = df_train_numeric['atm_group'].astype('object')
df_train_all_features = pd.get_dummies(df_train_numeric, drop_first=True)

C:\Users\User\AppData\Local\Temp\ipykernel_14400\2551875292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_numeric['atm_group'] = df_train_numeric['atm_group'].astype('object')
C:\Program Files\Anaconda\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(df_train_all_features, df_train['target'], test_size=0.25, random_state=42)

In [48]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [49]:
y_pred = model.predict(X_test)

In [50]:
r2_score(y_test, y_pred)

0.6979196957404209

In [51]:
rms = mean_squared_error(y_test, y_pred)**0.5

In [52]:
rms

0.04745792357547151

качество заметно улучшилось

# Пробую Ridge регрессию

In [64]:
from sklearn.linear_model import Ridge

In [65]:
ridge_params = {'alpha':list(np.arange(0.01, 100, 0.1))}

In [66]:
gscv = GridSearchCV(Ridge(), param_grid = ridge_params, cv = 10, scoring='r2').fit(X_train, y_train)

In [67]:
gscv.best_score_

0.690611533810242

In [68]:
gscv.best_estimator_

Ridge(alpha=86.61000000000001)

In [69]:
model_ridge = Ridge(alpha = 86.61000000000001)
model_ridge.fit(X_train, y_train)

Ridge(alpha=86.61000000000001)

In [70]:
y_pred = model_ridge.predict(X_test)

In [71]:
r2_score(y_test, y_pred)

0.6969928830024312

In [72]:
rms = mean_squared_error(y_test, y_pred)**0.5

In [73]:
rms

0.04753067065931728

Качество слегка ухудшилось

# Метод ближайших соседей

In [74]:
from sklearn.neighbors import KNeighborsRegressor

In [75]:
model = KNeighborsRegressor(n_neighbors=10) #лучший результат достигается с 10-ю соседями
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [76]:
y_pred = model.predict(X_test)

In [77]:
r2_score(y_test, y_pred)

0.5550742957209278

In [78]:
rms = mean_squared_error(y_test, y_pred)**0.5
rms

0.05759586112760437

Качество сильно хуже чем у обычной регрессии

# Метод опорных векторов

In [79]:
from sklearn.svm import SVR
regr = svm.SVR()
model = regr.fit(X_train, y_train)

In [80]:
у_pred = model.predict(X_test)

In [81]:
r2_score(y_test, y_pred)

0.5550742957209278

In [82]:
rms = mean_squared_error(y_test, y_pred)**0.5
rms

0.05759586112760437

Тоже хуже чем обычная регрессия

# Catboost

In [83]:
cat_features= ['regions','states','atm_group','cities']

In [84]:
df_train['atm_group'] = 'str' + df_train['atm_group'].astype(str)

In [85]:
df_for_catboost = df_train[['atm_group', 'distance_to_fast_food',
       'distance_to_clothes', 'distance_to_vending_parking',
       'distance_to_cafe', 'distance_to_pharmacy', 'distance_to_atm',
       'distance_to_cinema', 'distance_to_shoe_shop', 'distance_to_bank',
       'distance_to_supermarket', 'distance_to_restaurant',
       'distance_to_mobile_phone_shop', 'distance_to_convenience',
       'distance_to_vending_any', 'distance_to_bank_Сбербанк',
       'distance_to_bank_ВТБ', 'distance_to_bank_Альфа-Банк',
       'distance_to_bank_Росбанк', 'distance_to_atm_Сбербанк',
       'distance_to_atm_ВТБ', 'distance_to_atm_Альфа-Банк',
       'distance_to_atm_Росбанк', 'distance_to_retail',
       'distance_to_residential', 'distance_to_office',
       'distance_to_commercial', 'distance_to_detached',
       'distance_to_train_station', 'distance_to_apartments',
       'distance_to_house', 'distance_to_railway_halt',
       'distance_to_tram_stop', 'distance_to_bus_stop',
       'distance_to_railway_station', 'distance_to_airport',
       'distance_to_parking', 'distance_to_parking_underground',
       'cnt_apart_200m', 'cnt_banks_200m', 'cnt_atm_200m', 'regions',
       'states','cities']]

In [86]:
from catboost import CatBoostRegressor

In [87]:
model  = CatBoostRegressor(cat_features= cat_features,
                           eval_metric = 'RMSE',
                           random_seed = 42)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(df_for_catboost, df_train['target'], test_size=0.25, random_state=42)

In [91]:
model.fit(X_train, y_train)#, eval_set = (X_val, y_val))

Learning rate set to 0.051866
0:	learn: 0.0833585	total: 189ms	remaining: 3m 8s
1:	learn: 0.0807143	total: 224ms	remaining: 1m 51s
2:	learn: 0.0782274	total: 255ms	remaining: 1m 24s
3:	learn: 0.0758660	total: 291ms	remaining: 1m 12s
4:	learn: 0.0736472	total: 320ms	remaining: 1m 3s
5:	learn: 0.0715783	total: 348ms	remaining: 57.7s
6:	learn: 0.0696615	total: 378ms	remaining: 53.6s
7:	learn: 0.0679078	total: 413ms	remaining: 51.2s
8:	learn: 0.0663392	total: 446ms	remaining: 49.1s
9:	learn: 0.0648120	total: 475ms	remaining: 47s
10:	learn: 0.0633876	total: 504ms	remaining: 45.3s
11:	learn: 0.0620831	total: 538ms	remaining: 44.3s
12:	learn: 0.0608578	total: 565ms	remaining: 42.9s
13:	learn: 0.0597713	total: 595ms	remaining: 41.9s
14:	learn: 0.0588537	total: 606ms	remaining: 39.8s
15:	learn: 0.0578760	total: 641ms	remaining: 39.4s
16:	learn: 0.0569743	total: 670ms	remaining: 38.8s
17:	learn: 0.0561564	total: 700ms	remaining: 38.2s
18:	learn: 0.0553889	total: 733ms	remaining: 37.8s
19:	learn:

In [92]:
y_pred = model.predict(X_test)

In [95]:
r2_score(y_test, y_pred)

0.7418344809429056

In [96]:
rms = mean_squared_error(y_test, y_pred)**0.5
rms

0.043872930689280464

Catboost показал лучший результат